# Project Euler Problem Set in Python
### Problems 36 - 40

## Double-base palindromes
### Problem #36

The decimal number, 585 = 1001001001<sub>2</sub> (binary), is palindromic in both bases.

Find the sum of all numbers, less than one million, which are palindromic in base 10 and base 2.

(Please note that the palindromic number, in either base, may not include leading zeros.)

In [1]:
def is_double_base_pal(n: int) -> bool:
    """True if n is palindrome in base 2 and 10"""
    s = str(n)
    if s != s[::-1]: 
        return False
    s = bin(n)[2:]
    return s == s[::-1]
    
def sum_double_base_pals(limit: int) -> int:
    """sum of palindromes base 2&10 below limit"""
    return sum(n for n in range(limit) if is_double_base_pal(n))

In [2]:
assert sum_double_base_pals(1000000) == 872187

In [3]:
import itertools

def make_bit_pal(n:int, odd:bool) -> int:
    """generates a binary palindrome duplicating n; if odd the 'middle' bit is skiped"""
    p = n
    if odd: n >>= 1
    while n > 0:
        p = p << 1 | n & 1
        n >>= 1
    return p 

def is_b10_pal(n: int) -> bool:
    """True if n is a decimal palindrome"""
    s = str(n)
    return s == s[::-1]

def sum_2base_pals(limit: int) -> int:
    """sums double base palindromes below limit"""
    psum = 0
    for odd in (True, False):
        for n in itertools.count(1):
            p = make_bit_pal(n, odd)
            if p >= limit: break
            if is_b10_pal(p): psum += p
    return psum

In [4]:
assert sum_2base_pals(1000000) == 872187

#### Answer: 872187
---

## Truncatable primes
### Problem #37

The number 3797 has an interesting property. Being prime itself, it is possible to continuously remove digits from left to right, and remain prime at each stage: 3797, 797, 97, and 7. Similarly we can work from right to left: 3797, 379, 37, and 3.

Find the sum of the only eleven primes that are both truncatable from left to right and right to left.

##### NOTE: 2, 3, 5, and 7 are not considered to be truncatable primes.

In [5]:
class PrimeSieve():

    def __init__(self, n: int):
        """instantiates a sieve of size n"""
        ps = [True]*(n>>1)
        for i in range(3, int(n**0.5), 2):
            if not ps[(i-3)>>1]: continue
            for j in range(i*i, n, i):
                if j&1: ps[(j-3)>>1] = False
        self.size = n
        self.sieve = ps

    def is_prime(self, n:int) -> bool:
        """True if n is prime"""
        if n < 2: return False
        return self.sieve[(n-3)>>1] if n&1 else n==2
    
    def primes(self):
        """iterable over primes on the sieve"""
        return filter(self.is_prime, range(2, self.size))

In [6]:
def truncations(s: str) -> str:
    """iterates over truncations of s (excluding s)"""
    rs, ls = s, s
    while len(rs) > 1: # right
        rs = rs[:-1]
        yield rs
    while len(ls) > 1: # left
        ls = ls[1:]
        yield ls

In [7]:
class TruncablePrimes(PrimeSieve):
    
    def is_truncable_prime(self, n: int) -> bool:
        """True if n is a truncable prime"""
        if not self.is_prime(n): return False
        for k in map(int, truncations(str(n))): 
            if not self.is_prime(k): return False
        return True
    
    def truncables(self):
        """iterable over the truncables primes on the sieve"""
        return filter(self.is_truncable_prime, range(23, self.size, 2))  

In [8]:
assert sum(TruncablePrimes(999999).truncables()) == 748317

#### Answer: 748317
---

## Pandigital multiples
### Problem #38

Take the number 192 and multiply it by each of 1, 2, and 3:

&emsp; 192 × 1 = 192  
&emsp; 192 × 2 = 384  
&emsp; 192 × 3 = 576  

By concatenating each product we get the 1 to 9 pandigital, 192384576. We will call 192384576 the concatenated product of 192 and (1,2,3)

The same can be achieved by starting with 9 and multiplying by 1, 2, 3, 4, and 5, giving the pandigital, 918273645, which is the concatenated product of 9 and (1,2,3,4,5).

What is the largest 1 to 9 pandigital 9-digit number that can be formed as the concatenated product of an integer with (1,2, ... , n) where n > 1?

In [9]:
def max_pandigital_multiple():
    """largest 1-9 pandigital in the form k*(1,2,...,n)"""
    test_cases = ((9876,999,3), (987,99,4), (98,9,5), (9,0,6))
    pmax = 0
    for case in test_cases:
        for k in range(case[0], case[1], -1):
            sk = ''
            for n in range(1, case[2]):
                sk += str(n*k)
            if len(sk) == 9 == len(set(sk)) and '0' not in sk:
                pmax = max(pmax, int(sk))
                break
    return pmax

In [10]:
assert max_pandigital_multiple() == 932718654

#### Answer: 932718654
---

## Integer right triangles
### Problem #39

If p is the perimeter of a right angle triangle with integral length sides, {a,b,c}, there are exactly three solutions for p = 120.

{20,48,52}, {24,45,51}, {30,40,50}

For which value of p ≤ 1000, is the number of solutions maximised?

In [11]:
def max_perimeter(limit: int) -> int:
    """perimeter with maximun number of right angle triangles under limit"""
    h = {}
    m = limit//2 + 1
    for p in (a + b + (a*a + b*b)**0.5 
              for a in range(1, m) for b in range(a + 1, m - a)):
        k = int(p)
        if p == k:
            if k in h: h[k] +=1 
            else: h[k] = 1
    _, p = max((n,p) for p,n in h.items())
    return p

In [12]:
assert max_perimeter(1000) == 840

#### Answer: 840
---

## Champernowne's constant
### Problem #40

An irrational decimal fraction is created by concatenating the positive integers:

<p><center>0.12345678910<b>1</b>112131415161718192021...</center></p>

It can be seen that the 12<sup>th</sup> digit of the fractional part is 1.

If d<sub>n</sub> represents the n<sup>th</sup> digit of the fractional part, find the value of the following expression. 

<p><center>d<sub>1</sub> × d<sub>10</sub> × d<sub>100</sub> × d<sub>1000</sub> × d<sub>10000</sub> × d<sub>100000</sub> × d<sub>1000000</sub></center></p>

In [13]:
from functools import reduce
from operator import mul

# just go for it...
champ = ''.join(map(str, range(10**6)))
assert reduce(mul, (int(champ[10**i]) for i in range(6))) == 210

In [14]:
from itertools import count

# gets the digits wihout generating the whole thing
p, t, j = 1, 1, 1
for n in count(1):
    s = str(n)
    i, j = j, j + len(s)
    if i <= t < j:
        p *= int(s[t-i])
        t *= 10
    if t > 1000000: break
            
assert p == 210

#### Answer: 210
---